In [24]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import *

In [2]:
spark = (SparkSession.builder
         .appName("JDBC source")
         .master("yarn")
         .enableHiveSupport()
         .getOrCreate()
        )

2023-02-19 12:20:12,413 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-02-19 12:20:19,089 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [10]:
import configparser

In [17]:
config = configparser.RawConfigParser()

config.read('/opt/db_conn')
user_name = config.get('DB', 'user_name')
password = config.get('DB', 'password')
db_ip = config.get('DB', 'db_ip')

# Read from Postgresql

In [18]:
df_books = (spark.read.format("jdbc")
.option("driver", "org.postgresql.Driver")
.option("url", f"jdbc:postgresql://{db_ip}:5432/traindb")
.option("dbtable", "books")
.option("user", user_name)
.option("password", password)
.load())

In [19]:
df_books.show()

+---+--------------------+----------+---------+---------+--------------+------------+---------+------------+
| id|           book_name|      isbn|  book_id|    price|price_currency|rating_count|author_id|publisher_id|
+---+--------------------+----------+---------+---------+--------------+------------+---------+------------+
| 13|Madam Bovary (Cil...|6050948752|489127179|25.115735|           TRY|           5|  4098249|       46868|
| 22|Mai ve Siyah (Ele...|9750523533|492625951|    25.34|           TRY|          17|  3066057|       63217|
| 27|               Nutuk|9759914288|  9927355| 11.48147|           TRY|          23|  9705003|       46868|
| 34|              Devlet|9754734263|395307782|  27.9994|           TRY|           0|  8978000|       20709|
+---+--------------------+----------+---------+---------+--------------+------------+---------+------------+



In [6]:
df_books_query = (spark.read.format("jdbc")
.option("driver", "org.postgresql.Driver")
.option("url", "jdbc:postgresql://127.0.0.1:5432/traindb")
.option("query", "select * from books")
.option("user", "train")
.option("password", "Ankara06")
.load())

In [7]:
df_books_query.show()

+---+--------------------+----------+---------+---------+--------------+------------+---------+------------+
| id|           book_name|      isbn|  book_id|    price|price_currency|rating_count|author_id|publisher_id|
+---+--------------------+----------+---------+---------+--------------+------------+---------+------------+
| 13|Madam Bovary (Cil...|6050948752|489127179|25.115735|           TRY|           5|  4098249|       46868|
| 22|Mai ve Siyah (Ele...|9750523533|492625951|    25.34|           TRY|          17|  3066057|       63217|
| 27|               Nutuk|9759914288|  9927355| 11.48147|           TRY|          23|  9705003|       46868|
| 34|              Devlet|9754734263|395307782|  27.9994|           TRY|           0|  8978000|       20709|
+---+--------------------+----------+---------+---------+--------------+------------+---------+------------+



In [21]:
df_books_query.printSchema()

root
 |-- id: integer (nullable = true)
 |-- book_name: string (nullable = true)
 |-- isbn: long (nullable = true)
 |-- book_id: long (nullable = true)
 |-- price: double (nullable = true)
 |-- price_currency: string (nullable = true)
 |-- rating_count: integer (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- publisher_id: integer (nullable = true)



# Write to Postgresql

In [28]:
(df_books_query.withColumn("book_id", F.col("book_id").cast(StringType()))
 .write.format("jdbc")
.mode("append")
.option("driver", "org.postgresql.Driver")
.option("url", f"jdbc:postgresql://{db_ip}:5432/traindb")
.option("dbtable", "books_spark")
.option("user", user_name)
.option("password", password)
.save())


In [26]:
(spark.read.format("jdbc")
.option("driver", "org.postgresql.Driver")
.option("url", "jdbc:postgresql://127.0.0.1:5432/traindb")
.option("query", "select * from books_spark")
.option("user", "train")
.option("password", "Ankara06")
.load()).printSchema()

root
 |-- id: integer (nullable = true)
 |-- book_name: string (nullable = true)
 |-- isbn: long (nullable = true)
 |-- book_id: string (nullable = true)
 |-- price: double (nullable = true)
 |-- price_currency: string (nullable = true)
 |-- rating_count: integer (nullable = true)
 |-- author_id: integer (nullable = true)
 |-- publisher_id: integer (nullable = true)

